# チャレンジ用MIDIデータの作成

ベロシティをランダムにして、「L:R = 70:30」の割合でMODO DRUM用のmidiデータを作成する。

In [1]:
import numpy as np
import pretty_midi as midi

In [2]:
import import_ipynb
import functions as myfunc

myfunc.setup_seed(123)

importing Jupyter notebook from functions.ipynb


In [3]:
def make_notes(l_notes, r_notes, interval=2.0, duration=0.125):
    total_notes = l_notes + r_notes

    # ピッチデータ
    SNARE_L, SNARE_R = 38, 62
    labels = np.append(np.zeros(l_notes), np.ones(r_notes))  # 0/1のデータ
    labels = np.random.permutation(labels).astype(np.int32)  # シャッフル
    pitches = map(lambda x: SNARE_L if x == 0 else SNARE_R, labels)  # 0/1を音階に変換

    # ベロシティ
    velocities = np.random.randint(40, 128, total_notes)

    # Note On/Offタイミング（秒）
    points = np.arange(0, total_notes)
    notes_on = map(lambda x: x * interval, points)
    notes_off = map(lambda x: x * interval + duration, points)

    # Velocity, Pitch, Note On, Note Off のデータを組み合わせる
    notes = zip(velocities, pitches, notes_on, notes_off)
    return notes, labels


def save_midi(filename, notes):
    # ノート追加
    inst = midi.Instrument(program=0)
    for i, (vel, pitch, on, off) in enumerate(notes):
        note = midi.Note(velocity=vel, pitch=pitch, start=on, end=off)
        inst.notes.append(note)

    # MIDIファイルの保存
    data = midi.PrettyMIDI()
    data.instruments.append(inst)
    data.write(filename)
    print(f"File:{filename}, Notes:{i+1}")

In [4]:
# MIDIファイル作成
notes, labels = make_notes(70, 30)  # Snare L:R = 70:30
save_midi("./data/challenge.mid", notes)

# 正解ラベルをファイルに出力
line = "".join(map(str, labels))  # 1行の文字列に変換
myfunc.writelines("./data/challenge_true_labels.txt", [line])

# 正解ラベルの表示
print(f"True labels:{line}")

File:./data/challenge.mid, Notes:100
True labels:0110000010001000100101000110000100111100010001010000000100100100000100000000011000000101100010101010
